In [9]:
import numpy as np
import math

In [10]:
L,d_k,d_v=4,8,8
q=np.random.randn(L,d_k)
k=np.random.randn(L,d_k)
v=np.random.randn(L,d_v)

In [11]:
print("Q\n",q)
print("K\n",k)
print("V\n",v)

Q
 [[-0.70944123 -0.01773769  0.00660004  0.03448526  0.22175656  2.26415323
   1.93139     3.06905117]
 [ 0.38648036  0.8095378  -0.39726029  1.16352854 -1.56546819  0.11917732
   0.85248551 -1.98310562]
 [-0.92242717 -2.38675021 -1.28550749 -1.4020073  -1.33840008 -0.16222289
  -0.66943553 -1.00281251]
 [-0.03767562  0.26171853  0.44964634 -0.50946774 -0.69681987  0.60648336
   0.41978713 -0.22402015]]
K
 [[-0.15465855 -0.21578203 -0.77629568  0.35883392 -0.8689555  -0.78750826
   2.66052276  0.72965596]
 [ 0.47434482  1.84712089 -0.78525318  1.33067049 -1.31327505 -0.19759242
   0.04891668  0.96763674]
 [ 2.0372846   0.41433253 -0.34158634  1.39513177 -1.18161527 -1.85961313
  -0.38594565 -1.52652472]
 [ 1.25096607  0.2844642   0.06900961 -0.78687934  0.18154905 -0.52867144
   0.98693017 -0.96660375]]
V
 [[-1.25252297  0.14268162  0.91559593 -0.02155435  0.37997314 -0.59708641
  -1.3296059   2.03094207]
 [ 0.80542341  0.26978001 -1.58675757 -0.59494545  0.80059199 -0.69455677
  -0.9

Mulitply the query vector to the transpose of the of K vector. 

In [12]:
np.matmul(q,k.T)

array([[  5.52292208,   1.99701933, -11.30970006,  -3.13635568],
       [  2.57899011,   3.69397881,   7.20816671,   2.1817931 ],
       [ -0.06946941,  -4.91568898,  -0.71269844,  -0.66695832],
       [  0.49877028,   0.03357874,  -0.95717927,   0.64294007]])

The following 4*4 matrix tells how a particular word is related to the other word. In the first row the after the word's affinity with itself it has greatest affinity to the second word.

In [13]:
q.var(),k.var(),np.matmul(q,k.T).var()

(1.3465203815980025, 1.1421827853397775, 17.220820204786598)

As evident in the above values of variance q vector & k vector's variance is close to 1, however the variance of q*K.T vector is high. To reduce the variance self attention formula has the division of the product with sqrt(d_k)

In [16]:
scaled=(np.matmul(q,k.T)/math.sqrt(d_k))
q.var(),k.var(),scaled.var()

(1.3465203815980025, 1.1421827853397775, 2.152602525598324)

In [17]:
mask=np.tril(np.ones((L,L)))
mask

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

The following mask function will generate a triangular matrix. 
The 1's in the matrix ensure that the context is relevant to nearest words and not the succeeding ones.

In [18]:
mask[mask==0]=float("-inf")
mask[mask==1]=0

In [19]:
mask

array([[  0., -inf, -inf, -inf],
       [  0.,   0., -inf, -inf],
       [  0.,   0.,   0., -inf],
       [  0.,   0.,   0.,   0.]])

In [20]:
scaled + mask

array([[ 1.95264783,        -inf,        -inf,        -inf],
       [ 0.9118107 ,  1.30601873,        -inf,        -inf],
       [-0.02456115, -1.73795851, -0.25197695,        -inf],
       [ 0.17634192,  0.01187188, -0.33841398,  0.22731364]])

In [22]:
np.exp(scaled+mask)

array([[7.04732298, 0.        , 0.        , 0.        ],
       [2.48882496, 3.69144777, 0.        , 0.        ],
       [0.97573802, 0.17587909, 0.77726265, 0.        ],
       [1.19284585, 1.01194263, 0.7129001 , 1.2552235 ]])

In [24]:
np.sum(np.exp(scaled+mask))

19.329387571474136

In [26]:
np.sum(np.exp(scaled+mask),axis=1)

array([7.04732298, 6.18027274, 1.92887977, 4.17291208])

In [29]:
(np.exp(scaled+mask).T/np.sum(np.exp(scaled+mask),axis=1)).T

array([[1.        , 0.        , 0.        , 0.        ],
       [0.40270471, 0.59729529, 0.        , 0.        ],
       [0.50585736, 0.09118199, 0.40296065, 0.        ],
       [0.28585454, 0.24250274, 0.17083995, 0.30080277]])

Softmax function is used to convert the sclaed matrix into a probablitiy function

In [30]:
def softmax(x):
    return (np.exp(x).T/np.sum(np.exp(x),axis=-1)).T
def scaled_dot_product_attention(q,k,v,mask=None):
    d_k=k.shape[1]
    scaled=np.matmul(q,k.T)/math.sqrt(d_k)
    if mask is not None:
        scaled=scaled+mask
    attention=softmax(scaled)
    out=np.matmul(attention,v)
    return out.attention

array([[1.        , 0.        , 0.        , 0.        ],
       [0.40270471, 0.59729529, 0.        , 0.        ],
       [0.50585736, 0.09118199, 0.40296065, 0.        ],
       [0.28585454, 0.24250274, 0.17083995, 0.30080277]])